---
title: "Introduction to Metabolic Modelling"
subtitle: "Centre for Microbiome Research - Queensland University of Technology"
author: "Alexandre Areias Castro"
toc: true
bibliography: bibliography.bib
csl: https://www.zotero.org/styles/taylor-and-francis-national-library-of-medicine
---


***
In this tutorial:

- You will learn several concepts and methods within metabolic modelling, including:
    - Network reconstruction;
    - Single-model simulations and extensions;
    - Community modelling.
- Use different tools and methodologies for each step;
- Make changes or add lines to existing code to observe changes in results. Use this as a sandbox and <u>explore</u>!


In [ ]:
import pandas as pd
import micom
import reframed

# Reconstruction and model building

In this part:

- How a metabolic model originates from a genome;
- The main types of reconstruction;
- What tools to use.

***

According to ChatGPT, metabolic network reconstruction is:

"(...) a process where the complete set of metabolic reactions and pathways
in a microorganism, tissue, or organism are reconstructed and represented
in a computational model based on its genomic data. This model provides a
comprehensive framework for understanding the organism's metabolic network and
can be used to simulate its behavior under various conditions."

This is a fairly accurate answer. Now, let's look at how we can *actually* do this.

## 1 - Introduction

Metabolic reconstructions are achieved through essentially mapping genes,
reactions and metabolites into one cohesive and interconnected network known
as a **Genome-Scale Metabolic Model (GEM)**. In practical terms, it requires us to
obtain as much information from the genomic data so that through it, we can know
which reactions are related to each of the genes present in the genome. The way
we obtain this information is accomplised using 2 main approaches:

- Bottom-up
- Top-down

Bottom-up approaches are the most common and most used reconstruction
methodologies. They are comprised of a mapping of the genes to some sort
of annotation/database that extracts the proteins and reactions they
mediate/catalyze. Afterwards, they connect all the available reactions
and establish a "mock-up" metabolic network (also known as a scaffold)
and afterwards are curated to cover any missing reactions to the pathways.
Generally speaking, they are extremely good for generating models in a
manual/semi-automated way, but the more you automate them, the more the models
tend to lose **quality**. If you want to maintain quality, the ammount of work
needed to curate and fill the gaps exponentially increases. Examples of tools:
[ModelSEED](https://academic.oup.com/nar/article/49/D1/D575/5912569?login=false)
[@seaverModelSEEDBiochemistryDatabase2021], [merlin](https://academic.oup.com/nar/article/50/11/6052/6606174) [@capelaMerlinImprovedFramework2022] and
[gapseq](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-021-022
95-1) [@zimmermannGapseqInformedPrediction2021] (for gap-filling).

On the other hand, top-down reconstructions use the the scaffolds generated
in the first step and then compare them against a universal or complete
model with all the available reactions in every pathway from a database
and then extract the pathways they have. This methodology is much more
effecient in generating large numbers of models without compromising on
quality and needing manual curation, but bad for generating models for
*de novo* genomes. One example of tools that utilize this approach is
[CarveMe](https://pubmed.ncbi.nlm.nih.gov/30192979/) [@machadoFastAutomatedReconstruction2018].

![**Source:** Machado D, Andrejev S, Tramontano M, et al. Fast automated reconstruction of genome-scale metabolic models for microbial species and communities. Nucleic Acids Research. 2018;46(15):7542–7553.](data/images/gky537fig1.jpg)

## 2 - Model Definition

The file type used to represent metabolic networks as GEMs is the Systems
Biology Markup Language (SBML). To put it simply, these are XML files with
specific notation and rules that allow the grouping and connection between
**genes**, **reactions**, **metabolites** and their respective **units**. There
are a large number of conventions and versions constantly being released to
constantly make these models more updated and comprehensive. To see the specifications
for each version see [here](https://sbml.org/documents/specifications/).

Below is the example of a model for the central carbon metabolism of *E. coli*.
It is faily small in size and is the most widely used model for educational
purposes.


In [ ]:
#| echo: false

!cat data/models/e_coli_core.xml

## 3 - Testing

We will now test the *e_coli_cole* model by reading it and simulating microbial growth using **Flux-Balance Analysis (FBA)**:


In [ ]:
#| eval: false

!carve --refseq GCF_000026305.1 -o data/models/ecoli_ed1a.xml -g M9 -i M9 -v

Now let's test the model!


In [ ]:
from reframed import load_cbmodel

# Reading the model
model = load_cbmodel('data/models/ecoli_ed1a.xml')

# Summary of the model
model.summary()

# Simulation

In this part:

- You will learn how to perform flux balance analysis.
- You will use the ReFramed python library for metabolic modeling. You can check the online documentation for more details.

***

## 1- Running the simulation

The most simple thing you can do with a constraint-based model is to run a flux
balance analysis (FBA) simulation. This simulation method uses a stoichiometric
matrix to represent the metabolic network, where each row represents a
metabolite and each column represents a reaction. Afterwards, it calculates the
flow of metabolites in the network by solving a linear program with an objective
function that normally points towards microbial growth or the production of a
specific metabolite [@orthWhatFluxBalance2010].

> **INFO:** The simulation is using the default objective function (biomass maximization) and environmental conditions (aerobic growth in M9 minimal medium with glucose) that came pre-defined in this model.


In [ ]:
from reframed import FBA

solution = FBA(model)
print(solution)


In [ ]:
solution.show_values(pattern="R_EX", sort=True)


Questions to explore:

- How realistic are these growth values? How can they be more accurate (if possible)?
- Is *E. coli* growing at its maximum growth rate?

## 2 - Visualizing fluxes

Escher is a really nice tool for displaying fluxes in a metabolic map. It makes
your life easier, especially when comparing flux distributions after performing
some perturbations (such as gene deletions or changes in the growth medium).


In [ ]:
from reframed import fluxes2escher

fluxes2escher(solution.values)

## 3 - Environment and growth media

You can change growth conditions either by modifying the flux bounds of the reactions directly in the model or by supplying those constraints as an argument to the FBA simulation method.

Let's observe what happens if we remove oxygen uptake to simulate anaerobic growth:


In [ ]:
solution2 = FBA(model, constraints={'R_EX_o2_e':0})
print(solution2)


In [ ]:
solution2.show_values(pattern="R_EX", sort=True)


As expected, *E. coli* switched to a fermentation mode, which resulted in the secretion of fermentation products and a decrease in growth rate.

Again, we can see it better by displaying the flux distribution in a metabolic map.


In [ ]:
fluxes2escher(solution2.values)

## 4 - Comparing with experimental data

In order to validade the relevance of the usage of these methods, let's compare them with experimentally measured fluxes.

Gerosa and co-workers[@gerosaPseudotransitionAnalysisIdentifies2015] have measured fluxes in E. coli growing with different carbon sources.

![](data/images/gerosa2015.png)

Let's load the fluxomics data that has been stored as a CSV file...


In [ ]:
import pandas as pd
fluxomics = pd.read_csv('data/gerosa2015.csv', index_col=0)

fluxomics.sample(5) # print 5 random entries


We will constrain the model using only the respective uptake rate for each substrate and see how well it predicts the growth rate and all the other fluxes.

Unfortunately, our model only contains 5 of the 8 substrates used in the paper.


In [ ]:
uptake_reactions = {
    'Glucose': 'R_EX_glc__D_e',
    'Gluconate': 'R_EX_glcn_e',
    'Galactose': 'R_EX_gal_e',
    'Pyruvate': 'R_EX_pyr_e',
    'Glycerol': 'R_EX_glyc_e',
    'Succinate': 'R_EX_succ_e',
    'Acetate': 'R_EX_ac_e',
    'Fructose': 'R_EX_fru_e',
}

growth_rates = {
    'Glucose': 0.65,
    'Gluconate': 0.59,
    'Galactose': 0.18,
    'Pyruvate': 0.39,
    'Glycerol': 0.49,
    'Succinate': 0.51,
    'Acetate': 0.29,
    'Fructose': 0.49,
}


We need to remove glucose from the pre-defined medium, by setting the lower bound of the exchange reaction to zero:


In [ ]:
model.reactions.R_EX_glc__D_e.lb = 0


Now let's run simulations for all the five conditions.


In [ ]:
simulated = {}
print('Condition \tGrowth \tPredicted')

for condition, rxn_id in uptake_reactions.items():
    uptake_rate = fluxomics.loc[rxn_id, condition]
    solution = FBA(model, constraints={rxn_id: uptake_rate})
    simulated[f'{condition}_sim'] = solution.values
    print(f'{condition} \t{growth_rates[condition]} \t{solution.fobj}')
    
combined = pd.concat([fluxomics, pd.DataFrame(simulated)], axis=1, join='inner')


It seems that, in general, the model predicted similar growth rates to those that were measured.

> Why do you think this happened?

Now let's look at the same procedure with the *e_coli_core* model. Before that, we will have to reduce the number of carbon sources from 8 to 5 to accomodate the available exchange reactions in the model.


In [ ]:
uptake_reactions = {
    'Glucose': 'R_EX_glc__D_e',
    'Pyruvate': 'R_EX_pyr_e',
    'Succinate': 'R_EX_succ_e',
    'Acetate': 'R_EX_ac_e',
    'Fructose': 'R_EX_fru_e',
}

growth_rates = {
    'Glucose': 0.65,
    'Pyruvate': 0.39,
    'Succinate': 0.51,
    'Acetate': 0.29,
    'Fructose': 0.49,
}


In [ ]:
# Reading the model
model = load_cbmodel('data/models/e_coli_core.xml')

model.reactions.R_EX_glc__D_e.lb = 0

simulated = {}
print('Condition \tGrowth \tPredicted')

for condition, rxn_id in uptake_reactions.items():
    uptake_rate = fluxomics.loc[rxn_id, condition]
    solution = FBA(model, constraints={rxn_id: uptake_rate})
    simulated[f'{condition}_sim'] = solution.values
    print(f'{condition} \t{growth_rates[condition]} \t{solution.fobj}')
    
combined = pd.concat([fluxomics, pd.DataFrame(simulated)], axis=1, join='inner')


Now we observe that overall, the model predicted higher growth rates than what was measured.

> How can we explain the differences? What could be the variables at play here?

One of the limitations of FBA is that it does not predict overflow metabolism, unless we explicitly add additional constraints.

So let's now additionally constrain the acetate secretion rate as well and see if our predictions improve.


In [ ]:
simulated = {}
print('Condition \tGrowth \tPredicted')

for condition, rxn_id in uptake_reactions.items():
    constraints = {
        rxn_id: fluxomics.loc[rxn_id, condition],
        'R_EX_ac_e': fluxomics.loc['R_EX_ac_e', condition],
    }
    solution = FBA(model, constraints=constraints)
    simulated[f'{condition}_sim'] = solution.values
    print(f'{condition} \t{growth_rates[condition]} \t{solution.fobj}')
    
combined2 = pd.concat([fluxomics, pd.DataFrame(simulated)], axis=1, join='inner')


There is some improvement in the prediction of growth rates. But what about the fluxes? How well are they predicted, and does this also improve when we constrain acetate secretion?

The  plots below represent the measured vs predicted fluxes, before and after constraining acetate secretion.


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 5, figsize=(15, 6))

for i, condition in enumerate(uptake_reactions):
    combined.plot.scatter(condition, f'{condition}_sim', ax=axs[0,i])
    axs[0,i].plot([-20, 20], [-20, 20], 'k--', alpha=0.3)
    
    combined2.plot.scatter(condition, f'{condition}_sim', ax=axs[1,i])
    axs[1,i].plot([-20, 20], [-20, 20], 'k--', alpha=0.3)

fig.tight_layout()


<!-- ## 5 - Extensions of models -->

<!-- As you might have understood by now, the secret to accurate predictions lies in the constraints applied to the system. That is because by doing so, we reduce the **solution space** to the point where the predicted fluxes are extremely accurate. -->

<!-- ![**Source:** Orth JD, Thiele I, Palsson BØ. What is flux balance analysis? Nature biotechnology. 2010;28(3):245–248.](data/images/solution_space.png) -->

<!-- However, how do we reduce the solution space **and** shift it towards real flux values? -->

<!-- Currently we have 2 main options: -->

<!-- - We comprehensively define the growth media with highly precise fluxes -->
<!-- - We integrate omics data into the model -->

<!-- Defining a really accurate growth media is extremely dificult. First, because converting metabolite concentration into uptake fluxes is next to impossible. Therefore, most advances in this area have been towards extending and enhancing the models with various omics data. -->

<!-- There are currently a few tools that automatically integrate omics (mainly enzymatic/proteomics data) into a model. These processes generate what is commonly known as a **enzyme-constrained Genome-Scale Metabolic Model (ecGEM)**, with some small variances in both the process and the outcome. Some of the main tools and workflows are [ECMpy](https://github.com/tibbdc/ECMpy)[@maoECMpySimplifiedWorkflow2022], [GECKO](https://github.com/SysBioChalmers/GECKO/)[@chenReconstructionSimulationAnalysis2024], MOMENT[@adadiPredictionMicrobialGrowth2012] (sMOMENT->[AutoPACMEN](https://github.com/klamt-lab/autopacmen))[@bekiarisAutomaticConstructionMetabolic2020] and my own published on [GitHub](https://github.com/areias03/tmcom). -->
<!-- The process consists of 5 main stages: -->

<!-- 1. Expansion from a starting metabolic model to an ecModel structure -->
<!-- 2. Integration of enzyme turnover numbers into the ecModel structure -->
<!-- 3. Model tuning (or calibration) -->
<!-- 4. Integration of proteomics data into the ecModel -->
<!-- 5. Simulation and analysis of ecModels -->

<!-- Due to the complexity of the process, and mainly, our own time constraints we will just use a model generated by one of them, in this case the ... methodology -->


# Community Modeling

In this part:

- Build models for artificial communities
- Use the [MEWpy]() library for metabolic modelling. You can also read the documentation [here]().
- Analyze interactions and results from different simulation methods.

## 1 - Setting up

We will create a synthetic microbial consortium with two *E. coli* mutants growing in minimal medium. In one of the mutants we will knockout the glucose transporter and in the other we will knockout the ammonium transporter.


In [ ]:
from reframed import load_cbmodel
wildtype = load_cbmodel('data/models/e_coli_core.xml')

Now we create our two mutants (`glc_ko` and `nh4_ko`):


In [ ]:
glc_ko = wildtype.copy()
glc_ko.id = 'glc_ko'
glc_ko.set_flux_bounds('R_GLCpts', 0, 0)

In [ ]:
nh4_ko = wildtype.copy()
nh4_ko.id = 'nh4_ko'
nh4_ko.set_flux_bounds('R_NH4t', 0, 0)

### Comparing models

Community models require that metabolites have the same identifiers across all models. MEWpy offers some functions tho that end, computing the metabolites, reactions and uptakes overlaps between a list models.


In [ ]:
from mewpy.com import *
mets, rxns, over = jaccard_similarity_matrices([glc_ko, nh4_ko])

In [ ]:
mets

In [ ]:
rxns

In [ ]:
over

### Building communities

MEWpy has some basic functionality for working with microbial communities, one is the `CommunityModel` class to create microbial communities from a list of models of individual species:


In [ ]:
from reframed import Community
community = Community('ecoli_pair', [glc_ko, nh4_ko])

This community model ignores the environmental conditions that were specified in the original models (since these could be very different).

To make our life easier, we will extract the nutrient composition specified in the wild-type model to use later.


In [ ]:
from reframed import Environment

M9 = Environment.from_model(wildtype)

print(f"Environment compounds: {', '.join(M9.get_compounds())}")

## 2 - Simulation using FBA

A very simple way to simulate a microbial community is to merge the individual models into a single model that mimics a "super organism", where each microbe lives inside its own compartment, and run a (conventional) FBA simulation for this super organism.


In [ ]:
from reframed import FBA

super_oganism = community.merged_model
solution = FBA(super_oganism, constraints=M9)

print(solution)
solution.show_values(pattern='R_EX')

We can see that the model predicts a growth rate (total biomass per hour) similar to the wild-type, with an efficient consumption of glucose and ammonia that results in respiratory metabolism.

But what is each organism doing, and are both organisms actually growing at the same rate?


In [ ]:
solution.show_values(pattern='BIOMASS', sort=True)

Let's print the all non null fluxes by organism:


In [ ]:
solution.show_values(pattern='_glc_ko', sort=True)

In [ ]:
solution.show_values(pattern='_nh4_ko', sort=True)

Actually it seems that only one of the organisms is growing while the other has an active metabolism (it exchanges metabolites with the environment and with the other organism) performing the role of a bioconverter, but none of the flux is used for growth.

> Do you think this would be a stable consortium ?


## 3 - Simulation using SteadyCom

SteadyCom[@chanSteadyComPredictingMicrobial2017] is a recent community simulation method that takes into account the fact that to reach a stable composition the organisms need to grow at the same specific growth rate (1/h), which means that the absolute growth rate (gDW/h) of each organism is proportional to its abundance at steady-state (gDW).

Let's simulate the same community using SteadyCom:


In [ ]:
from reframed import SteadyCom
steady= SteadyCom(community, constraints=M9, solver = 'cplex')

In this case the solution object shows the overall community growth rate and the relative abundance of each species:


In [ ]:
print(steady)

The `solution` object for community simulations implements a few additional features, such as enumerating all the cross-feeding interactions:


In [ ]:
steady.cross_feeding(as_df=True).dropna().sort_values('rate', ascending=False)

We can plot the fluxes of each mutant in a map to help with interpretation of the results:


In [ ]:
fluxes2escher(steady.internal['glc_ko'])

In [ ]:
fluxes2escher(steady.internal['nh4_ko'])

## 4 - Explore alternative solutions

## 5 - Interaction Analysis (SMETANA)

# References

::: {#refs}
:::